In [8]:
from fastapi import FastAPI, UploadFile, File
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import io
from PIL import Image

app = FastAPI()

# Cargar el modelo
model = load_model('./modelapi_shitty.h5')

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    # Leer la imagen
    contents = await file.read()
    img = Image.open(io.BytesIO(contents))
    img = img.resize((256, 256))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Hacer la predicción
    prediction = model.predict(img_array)
    predicted_class = int(prediction[0][0] > 0.5)

    return {"prediction": predicted_class}

# Ejecutar el servidor con: uvicorn main:app --reload